In [2]:
import importlib
import numpy as np
import cv2 as cv
from PIL import Image
import lk_util
import lukas_kanade 
import yaml
import time

importlib.reload(lk_util)
importlib.reload(lukas_kanade)



<module 'lukas_kanade' from '/home/hannahcl/Documents/fagV23/Feature-tracking/optical_flow/lukas_kanade.py'>

In [3]:
# -- Import video frames

cap = cv.VideoCapture('https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4')


In [4]:

# Take first frame 
ret, old_frame = cap.read()
target_image = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

# #Find corners with goodFeaturesToTrack
# feature_params = dict( maxCorners = 100,
#                        qualityLevel = 0.3,
#                        minDistance = 7,
#                        blockSize = 7 )
# initial_feature_detections_positions = cv.goodFeaturesToTrack(target_image, mask = None, **feature_params)
# initial_feature_detections = np.array([[*p] for p in initial_feature_detections_positions.reshape(-1,2)])


#  Or, Use FAST feature detector to detect initial features
fast = cv.FastFeatureDetector_create(threshold=100)
initial_feature_detections = fast.detect(target_image, None)
initial_feature_detections = np.array([p.pt for p in initial_feature_detections])

print(initial_feature_detections)



In [5]:


# Initialize LucasKanade with configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

lk = lukas_kanade.LucasKanade(config)
lk.target_image = target_image
lk.get_initial_feature_detections(initial_feature_detections) 

time_recodring = []

while(True):

    if lk.i_frame >= lk.n_frames -1:
        break

    ret, frame = cap.read()

    start_time = time.perf_counter()
    lk.find_tracks_in_target_image(frame)
    end_time = time.perf_counter()

    elapsed_time_ms = (end_time - start_time) * 1000
    time_recodring.append(elapsed_time_ms)

    if lk.show_and_save_tracks == True:
        lk.show_and_save_tracked_features()

time_recodring = np.array(time_recodring)
average_time = np.average(time_recodring)       
print(f"Over {lk.i_frame}s, it took {average_time} milliseconds on avarage to track {lk.n_features} features.")

In [ ]:

lk.compare_tracks_to_ground_truth()   

cv.waitKey()
cv.destroyAllWindows()




dogbox arctan
Frame 3 and track 0
gt: [237.51762 150.13889], estimated: [349.48970406  18.65189955], error: [111.97208406 131.48699045]
Frame 3 and track 1
gt: [301.1077 103.7322], estimated: [350.2669365   22.38488908], error: [49.1592365  81.34731092]


IndexError: index 2 is out of bounds for axis 0 with size 2

: 